# [Advent of Code 2021](https://adventofcode.com/2021)

Partially inspired by [Norvig's pytudes](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb), although I probably won't be using the same style.

Goals
------
* Try to finish all gold stars on day of puzzle
* Learn new python features
* Investigate github copilot

In [163]:
# Imports

from typing import Callable, TypeVar

T = TypeVar('T')

# Helper functions

def data(day: int, parser: Callable[[str], T]) -> list[T]:
    with open(f"../data/2021/day{day}.txt") as f:
        return [parser(line.strip()) for line in f.readlines()]

## Day 1

Determine number of increments in list

In [13]:
data1 = data(1, int)

In [14]:
def day1(data: list[int]) -> int:
  count, current = 0, data[0]
  for i in data[1:]:
    if i > current:
      count += 1
    current = i
  return count

day1(data1)

1688

In [48]:
def win3(data: list[int]) -> list[int]:
  for i in range(len(data) - 2):
    yield sum(data[i:i+3])

day1(list(win3(data1)))

1728

## Day 2

Maintain sum of depth and length

In [21]:
data2 = data('2', str)

In [22]:
def day2(data: list[str]) -> int:
  depth, length = 0, 0
  for i in data:
    mode, amount = i.split(' ')
    amount = int(amount)
    match mode:
      case 'forward':
        length += amount
      case 'down':
        depth += amount
      case 'up':
        depth -= amount
  return depth * length

day2(data2)

1451208

In [26]:
def day2_2(data: list[str]) -> int:
  depth, length, aim = 0, 0, 0
  for i in data:
    mode, amount = i.split(' ')
    amount = int(amount)
    match mode:
      case 'forward':
        length += amount
        depth += amount * aim
      case 'down':
        aim += amount
      case 'up':
        aim -= amount
  return depth * length

day2_2(data2)

1620141160

## Day 3

Can probably be solved with bit tricks, but I'll just implement them as array problems.

In [176]:
data3 = data('3', str)

def binlist2int(l: list[int]) -> int:
  return int("".join(str(i) for i in l), 2)

In [177]:
def day3(data: list[int]):
  total = len(data)
  size = len(data[0])
  sums = [0] * size
  for binary in data:
    for i, v in enumerate(binary):
      sums[i] += int(v)
  
  gamma = [0 if total - i > i else 1 for i in sums]
  epsilon = [i ^ 1 for i in gamma]
  return binlist2int(gamma) * binlist2int(epsilon)

day3(data3)

2954600

In [179]:
def find_boundary(data: list[str], start: int, end: int, index: int) -> int:
  # TODO: this can be implemented as binary search
  for i, v in enumerate(data[start:end+1]):
    if v[index] == '1':
      return start + i
  return start

def scrubber(data: list[str], lcd: bool) -> int:
  # Run "binary search"
  low, high = 0, len(data) - 1
  for target in range(len(data[0])):
    index = low + ((high - low + 1) // 2)
    item = data[index][target]

    if lcd and item == '0':
      # Select all '1's in the bottom half
      low = find_boundary(data, index, high, target)
    elif lcd and item == '1':
      # Select all '0's in the top half
      high = find_boundary(data, low, index, target) - 1
    elif item == '0':
      # Select all '0's from the top through to the bottom half
      high = find_boundary(data, index, high, target) - 1
    else:
      # Select all '1' starting from the top half to the end
      low = find_boundary(data, low, index, target)
    
    if low == high:
      break

  return data[low]

def day3_2(data: list[str]) -> int:
  data = sorted(data)
  oxygen = scrubber(data, False)
  c02 = scrubber(data, True)
  return binlist2int(oxygen) * binlist2int(c02)

day3_2(data3)

1662846

## Day 4


In [178]:
# Scratch